In [2]:
import nibabel as nib
import numpy as np
import os

def check_geometry_compatibility(image_path, mask_path):
    # Cargar imágenes
    img_nii = nib.load(image_path)
    mask_nii = nib.load(mask_path)
    
    print(f"--- Comprobando: {os.path.basename(image_path)} ---")
    
    # 1. Comprobar Dimensiones
    shape_match = img_nii.shape == mask_nii.shape
    print(f"Shape Match: {shape_match} | Img: {img_nii.shape}, Mask: {mask_nii.shape}")
    
    # 2. Comprobar Affine (Orientación, Posición, Escala)
    # Usamos allclose porque pueden haber diferencias infinitesimales por punto flotante
    affine_match = np.allclose(img_nii.affine, mask_nii.affine, atol=1e-5)
    
    if affine_match:
        print("Affine Match: CORRECTO")
    else:
        print("Affine Match: ERROR")
        print("Diferencia máxima en matriz:", np.max(np.abs(img_nii.affine - mask_nii.affine)))
        print("Affine Imagen:\n", img_nii.affine)
        print("Affine Máscara:\n", mask_nii.affine)


img_file = "/media/guest/PORT-DISK/Practicas/MicroBleeds/nnUNet_raw_data/TaskVALDO/imagesTr/sub-101_0000.nii.gz"

mask_file = "/media/guest/PORT-DISK/Practicas/MicroBleeds/nnUNet_raw_data/TaskVALDO/labelsTr/sub-101.nii.gz"

check_geometry_compatibility(img_file, mask_file)

--- Comprobando: sub-101_0000.nii.gz ---
Shape Match: True | Img: (512, 512, 35), Mask: (512, 512, 35)
Affine Match: CORRECTO


In [ ]:
import nibabel as nib
import numpy as np
import os
import glob

# ================= CONFIGURACIÓN =================
# Ajusta esta ruta a la carpeta raíz de tu dataset en nnUNet_raw
# Ejemplo: /home/user/nnUNet_raw/Dataset501_Valdo
base_folder = "/media/guest/PORT-DISK/Practicas/MicroBleeds/nnUNet_raw_data/TaskVALDO"

images_dir = os.path.join(base_folder, "imagesTr")
labels_dir = os.path.join(base_folder, "labelsTr")
# =================================================

def check_dataset_health(images_dir, labels_dir):
    # Obtener lista de máscaras (labels)
    mask_files = sorted(glob.glob(os.path.join(labels_dir, "*.nii.gz")))
    
    if not mask_files:
        print("No se encontraron archivos en labelsTr. Revisa la ruta.")
        return

    problematic_cases = []
    
    print(f"Analizando {len(mask_files)} casos...\n")

    for mask_path in mask_files:
        mask_name = os.path.basename(mask_path)
        case_id = mask_name.replace(".nii.gz", "")
        
        # Construir nombre esperado de la imagen (nnUNet añade _0000)
        image_name = f"{case_id}_0000.nii.gz"
        image_path = os.path.join(images_dir, image_name)
        
        # 1. Comprobar existencia del archivo de imagen
        if not os.path.exists(image_path):
            print(f"FALTA IMAGEN: Para la máscara {mask_name}, no existe {image_name}")
            problematic_cases.append((case_id, "Missing Image"))
            continue

        try:
            # Cargar archivos (sin leer todos los datos a memoria aún para ir rápido)
            mask_nii = nib.load(mask_path)
            img_nii = nib.load(image_path)
            
            # 2. Comprobar Dimensiones (Shape)
            if img_nii.shape != mask_nii.shape:
                print(f"SHAPE MISMATCH en {case_id}: Img {img_nii.shape} != Mask {mask_nii.shape}")
                problematic_cases.append((case_id, "Shape Mismatch"))
                continue
            
            # 3. Comprobar Affine (Geometría)
            if not np.allclose(img_nii.affine, mask_nii.affine, atol=1e-5):
                print(f"AFFINE MISMATCH en {case_id}")
                # Mostrar diferencia para depurar
                diff = np.max(np.abs(img_nii.affine - mask_nii.affine))
                print(f"   Dif. máxima: {diff:.6f}")
                problematic_cases.append((case_id, "Affine Mismatch"))
                continue
                
            # 4. Comprobar corrupción de datos (NaNs)
            # Solo leemos los datos si pasa las pruebas anteriores
            if np.isnan(np.sum(mask_nii.get_fdata())):
                print(f"CORRUPCIÓN (NaNs) en la máscara: {case_id}")
                problematic_cases.append((case_id, "NaNs in Mask"))
                continue

        except Exception as e:
            print(f"ERROR LECTURA en {case_id}: {str(e)}")
            problematic_cases.append((case_id, "File Read Error"))

    # --- RESUMEN FINAL ---
    print("\n" + "="*30)
    print("RESUMEN DEL DIAGNÓSTICO")
    print("="*30)
    
    if len(problematic_cases) == 0:
        print("TODO PERFECTO. Los 72 casos tienen geometría coincidente.")
        print("Si nnUNet sigue fallando, revisa tu dataset.json o memoria RAM.")
    else:
        print(f"Se encontraron {len(problematic_cases)} casos problemáticos:\n")
        for case, reason in problematic_cases:
            print(f"  • {case}: {reason}")

# Ejecutar
check_dataset_health(images_dir, labels_dir)

Analizando 72 casos...

AFFINE MISMATCH en sub-103
   Dif. máxima: 0.000011
AFFINE MISMATCH en sub-107
   Dif. máxima: 0.000152
AFFINE MISMATCH en sub-109
   Dif. máxima: 0.000012
AFFINE MISMATCH en sub-111
   Dif. máxima: 0.000012

RESUMEN DEL DIAGNÓSTICO
Se encontraron 4 casos problemáticos:

  • sub-103: Affine Mismatch
  • sub-107: Affine Mismatch
  • sub-109: Affine Mismatch
  • sub-111: Affine Mismatch
